In [7]:
import os
import sys
import json
import random
import pandas as pd
from tqdm import tqdm
from typing import List

random.seed(11)

Загрузим данные с https://www.kaggle.com/coolonce/recipes-and-interpretation-dim

In [2]:
df = pd.read_csv("../all_recepies_inter.csv", sep="\t",
                  usecols=["name", "Инструкции"]).rename(
                      columns={"Инструкции": "instructions"})

In [3]:
df.head()

,name,instructions
0,рассольник классический с перловкой и солеными...,Подготовить указанные ингредиенты для приготов...
1,Суп пюре из белокочаной капусты,"Необходимые ингредиенты\r\nНарезаем лук, морко..."
2,Постные щи из квашеной капусты,"Честно признаюсь, у меня не было репы на момен..."
3,Тюря- простой суп быстро и вкусно,"\r\nНачинаем мы приготовление тюри с того, что..."
4,Фасолевый суп из красной фасоли,Подготовить ингредиенты. Для приготовления суп...


## Формируем датасет для обучения модели.

In [4]:
dialoges = []
for i in tqdm(range(len(df))):
    dialog_format = {
    "messages": [
    {"role": "user", "content": df.iloc[i]['name']},
    {"role": "bot", "content": df.iloc[i]['instructions']}
    ],
     "source": "alpaca"
    }
    dialoges.append(dialog_format)

  0%|          | 0/27884 [00:00<?, ?it/s]

100%|██████████| 27884/27884 [00:01<00:00, 23418.79it/s]


Чтобы уменьшить время обучения немного возьмем только первые 10000 рецептов.

In [8]:
random.shuffle(dialoges)
dialoges = dialoges[:10000]

### Разделение датасета на тренировочную и тестовую выборки.

In [9]:
def train_test_split(dataset, val_percent):
    random.shuffle(dataset)
    val_len = int(len(dataset) * val_percent)
    return dataset[val_len:], dataset[:val_len], 

In [10]:
train_dataset, test_dataset = train_test_split(dialoges, 0.15)
len(train_dataset), len(test_dataset)

(8500, 1500)

### Запись датасета на диск.

In [14]:
train_dataset_dir = '../datasets'
os.makedirs(train_dataset_dir, exist_ok=True)

In [15]:
def save_jsonl(dicts, file_name):
    with open(file_name, "w", encoding="utf-8") as w:
        for d in tqdm(dicts):
            w.write(json.dumps(d, ensure_ascii=False) + "\n")

In [16]:
save_jsonl(train_dataset, os.path.join(train_dataset_dir, 'train_recepies.jsonl'))
save_jsonl(test_dataset, os.path.join(train_dataset_dir, 'test_recepies.jsonl'))

100%|██████████| 1500/1500 [00:00<00:00, 61018.71it/s]
